In [214]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [215]:
# High-level imports test
import pydrake
import torch
import numpy as np

In [216]:
torch.compiled_with_cxx11_abi()

False

## Test 1: pytorch gradients -> np.array of pydrake.AutoDiffXd's

In [217]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [218]:
lin = nn.Linear(5, 3)    # maps from R^5 to R^3, parameters A, b
data = torch.from_numpy(np.random.randn(2, 5).astype(np.float32)) # data is 2x5.  A maps from 5 to 3... can we map "data" under A?
print(lin(data))
# print(F.relu(lin(data)))
np.random.randn(2, 5).dtype

tensor([[ 0.1455, -0.4936, -0.4120],
        [-0.6613, -0.2842, -0.1649]], grad_fn=<AddmmBackward>)


dtype('float64')

In [219]:
torch.randn(2, 5)

tensor([[-1.1948,  0.0250, -0.7627,  1.3969, -0.3245],
        [ 0.2879,  1.0579,  0.9621,  0.3935,  1.1322]])

In [180]:
import pydrake
import pydrake.autodiffutils
from pydrake.autodiffutils import AutoDiffXd

x = AutoDiffXd(1.5)
arr = np.ndarray((3,), buffer=np.array([1,2,3]), dtype=np.float64)
y = AutoDiffXd(1.5, arr)
for var in (x, y):
    print(var.value(), var.derivatives())

(1.5, array([], dtype=float64))
(1.5, array([0., 0., 0.]))


## Test 2: Implement a pydrake LeafSystem and get pytorch ff bits working

This is a great reference!
https://github.com/RobotLocomotion/drake/blob/master/bindings/pydrake/systems/test/custom_test.py

In [183]:
from NNSystem import NNSystem
from NNTestSetup import NNTestSetup

In [ ]:
# NOT passing in a pytorch net.
nn_test_setup = NNTestSetup()
nn_test_setup.RunSimulation()

In [210]:
# Giving NNSystem a pytorch net.
class MLP(nn.Module):
    def __init__(self, layer_norm=False):
        super(MLP, self).__init__()
        self.layer_norm = layer_norm

        self.l1 = nn.Linear(4, 64)
        self.ln1 = nn.LayerNorm(64)
        self.tanh1 = F.tanh
        self.l2 = nn.Linear(64, 64)
        self.ln2 = nn.LayerNorm(64)
        self.tanh2 = F.tanh
        self.l3 = nn.Linear(64, 3)
    
    def forward(self, x):
        x = self.l1(x)
        if self.layer_norm: x = self.ln1(x)
        x = self.tanh1(x)
        x = self.l2(x)
        if self.layer_norm: x = self.ln2(x)
        x = self.tanh2(x)
        x = self.l3(x)
        return x
    
nn_test_setup = NNTestSetup(pytorch_nn_object=MLP())
nn_test_setup.RunSimulation()

## Test 3: Propagate autodiff bits in the LeafSystem. May require dropping down into c++

In [ ]:
net = MLP()
list(net.modules())[1].in_features
# list(net.modules())[-1].out_features

In [ ]:
x = torch.randn(1, 4, requires_grad=True)
out = net.forward(x)
out

In [ ]:
# net.zero_grad()
# out.backward(torch.ones(3, 1))
out.backward()

In [ ]:
print(x.grad)